# LAB 5: Image search using CLIP

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/xlab-recommendation/blob/notebook/05_CLIP_image_search.ipynb)

* Dataset ref: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/overview
    * รูปภาพที่ใช้เป็น dataset ภายใน notebook นี้ เป็นรูปจาก H&M personalized fashion recommendations ที่สุ่มเลือกมา แล้วนำไป resize
    * ซึ่งฝากไว้บน google drive: 

* Objectives
    * ช่วยลูกค้าค้นหาเสื้อผ้าในร้าน/website จากรูปที่ลูกค้าให้มา

* Notes
    * openai-clip: https://github.com/openai/CLIP
    * faiss: https://github.com/facebookresearch/faiss/wiki

In [ ]:
# install library

! pip install torch ftfy regex tqdm numpy
! pip install openai-clip
! pip install gradio
! pip install faiss-cpu
! pip install gdown

In [24]:
# import essential library

import os
import os.path as op
from PIL import Image

import numpy as np
import torch
from tqdm import tqdm

import clip
import faiss

In [25]:
# set running device to cpu

device = "cpu"

In [26]:
# see openai-clip available pre-train model

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [27]:
# load Vit-B/32 model

model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# download pre-select dataset from shared google drive

import gdown

url = "https://drive.google.com/drive/folders/1eFmm2TUrsPUzPWRP_sE3CnJc6g-La3rC?usp=drive_link"
gdown.download_folder(url, use_cookies=False)

In [28]:
# load dataset

dataset_path = op.join(os.getcwd(), "HM_sample_dataset/")

# create list of all filename in dataset folder

all_image_name = os.listdir(dataset_path)

In [ ]:
# create embeddings vector using FAISS

index = faiss.IndexFlatL2(512) # dimension of 1 embedding decoded from CLIP model is 512

In [30]:
# encode dataset
dataset_embeddings = []

for image_name in tqdm(all_image_name):
    with torch.no_grad():
        image = preprocess(Image.open(op.join(dataset_path,image_name))).unsqueeze(0).to(device)
        dataset_embeddings.append(model.encode_image(image).numpy(force=True)[0].astype('float32'))

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:03<00:00,  5.38it/s]


In [ ]:
# add embeddings into faiss vector

dataset_embeddings_np = np.array(dataset_embeddings) # change to numpy array for FAISS 
index.add(dataset_embeddings_np)

print(index.ntotal) # number of images embeddings store in dataset vector

In [ ]:
import gradio as gr

def recommend_similar_image(image_path):
    print(f"get image path {image_path}")

    test_image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        test_embeddings = model.encode_image(test_image).numpy(force=True)[0].astype('float32')
        test_embeddings = np.array([test_embeddings])

    recommend_number = 4 # number of recommendations
    square_distance, image_index = index.search(test_embeddings,k)
    print(image_index)
    print(square_distance)
    
    print("Opening Images...")
    recommended_images = [(Image.open(dataset_path + all_image_name[image_index[0][i]]), f"Recommended Rank {i+1}") for i in range(recommend_number)]
    return recommended_images

demo = gr.Interface(
    fn=recommend_similar_image,
    inputs=gr.Image(type="filepath"),
    outputs= gr.Gallery(),
)

demo.launch()